<a href="https://colab.research.google.com/github/Cachipa/Topicos-3/blob/main/AS02_Representa%C3%A7%C3%A3o_Textual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Bibliotecas usadas, carrega os dados e preprocessa eles**

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import fetch_20newsgroups
from nltk import bigrams
import numpy as np
from gensim.models import Word2Vec
import re
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import pandas as pd

# Carregar os dados
newsgroups_train = fetch_20newsgroups(subset='train')

# Tokenização
tokenized_texts = [doc.split() for doc in newsgroups_train.data]

# Os dados utilizados são restringidos por essa variavel porque o colab nao aguenta tudo, para aumentar ou diminuir o numero de textos usados altere o numero abaixo
tokenized_texts_first_20 = tokenized_texts[:20]

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

string_texts = [' '.join(doc) for doc in tokenized_texts_first_5]

# Preprocessamento
def preprocess(text):
    text = remove_accents(text)
    text = re.sub('\.(?!(\$[^. ])|\d)', '', text)
    text = re.sub('(?<!\d)[.;!?\'\(\)#:-](?!\d)', '', text)
    text = re.sub(' +', ' ', text)
    text = text.lower()
    return text

preprocessed_texts = [preprocess(text) for text in string_texts]

**OneHotEncoder**

In [ ]:
one_hot_vectorizer = CountVectorizer(binary=True)
one_hot_encoded = one_hot_vectorizer.fit_transform(preprocessed_texts)
np.savetxt('20News_01.txt', one_hot_encoded.toarray(), fmt='%d')

**Count Vectors**

In [ ]:
count_vectorizer = CountVectorizer()
count_vectors = count_vectorizer.fit_transform(preprocessed_texts)
terms = count_vectorizer.get_feature_names_out()
np.savetxt('20News_02.txt', count_vectors.toarray(), fmt='%d')

**TF-IDF**

In [ ]:
tfidf_vectorizer = TfidfTransformer()
tfidf_vectors = tfidf_vectorizer.fit_transform(count_vectors)
np.savetxt('20News_03.txt', tfidf_vectors.toarray(), fmt='%.3f')

**n-grams (2-grams)**

In [ ]:
ngram_vectorizer = CountVectorizer(ngram_range=(2, 2))
ngram_vectors = ngram_vectorizer.fit_transform(preprocessed_texts)
np.savetxt('20News_04.txt', ngram_vectors.toarray(), fmt='%d')

**Co-occurrence Vectors (Context Window = 1)**

In [ ]:
co_occurrence_matrix2 = (count_vectors.T * count_vectors)
np.savetxt('20News_05.txt', co_occurrence_matrix2.toarray(), fmt='%d')

**Word2Vec**

In [ ]:
word2vec_model = Word2Vec(preprocessed_texts, vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.wv.save_word2vec_format('20News_06.txt', binary=False)